# Classification of Mines and Rocks using Sonar Data #

## Preface ##

This is an experiment with Deep Learning, to develop my understanding of how to best use it to solve problems.

The dataset can be found [here](http://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks)).

In [3]:
import numpy as np
import pandas as pd

df = pd.read_csv("./data/sonar.csv", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


### Understanding the Data ###

Each row in the dataset represents one detected element (either a rock or mine).

The first 60 columns (0-59) are the energies "within a particular frequency band, integrated over a certain period of time."

#### Frequency what? ####

From what I understand, a frequency domain is the distribution of signal over a range of frequencies. It records the number of times a type of peak or variation occurs throughout the data. A frequency band is an interval within this domain. The values in the first 60 columns capture the energy of the signal within a band.


The last column is the classification: M for mine and R for rock.

In [4]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
count,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,...,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000
mean,0.029164,0.038437,0.043832,0.053892,0.075202,0.104570,0.121747,0.134799,0.178003,0.208259,...,0.016069,0.013420,0.010709,0.010941,0.009290,0.008222,0.007820,0.007949,0.007941,0.006507
std,0.022991,0.032960,0.038428,0.046528,0.055552,0.059105,0.061788,0.085152,0.118387,0.134416,...,0.012008,0.009634,0.007060,0.007301,0.007088,0.005736,0.005785,0.006470,0.006181,0.005031
min,0.001500,0.000600,0.001500,0.005800,0.006700,0.010200,0.003300,0.005500,0.007500,0.011300,...,0.000000,0.000800,0.000500,0.001000,0.000600,0.000400,0.000300,0.000300,0.000100,0.000600
25%,0.013350,0.016450,0.018950,0.024375,0.038050,0.067025,0.080900,0.080425,0.097025,0.111275,...,0.008425,0.007275,0.005075,0.005375,0.004150,0.004400,0.003700,0.003600,0.003675,0.003100
50%,0.022800,0.030800,0.034300,0.044050,0.062500,0.092150,0.106950,0.112100,0.152250,0.182400,...,0.013900,0.011400,0.009550,0.009300,0.007500,0.006850,0.005950,0.005800,0.006400,0.005300
75%,0.035550,0.047950,0.057950,0.064500,0.100275,0.134125,0.154000,0.169600,0.233425,0.268700,...,0.020825,0.016725,0.014900,0.014500,0.012100,0.010575,0.010425,0.010350,0.010325,0.008525
max,0.137100,0.233900,0.305900,0.426400,0.401000,0.382300,0.372900,0.459000,0.682800,0.710600,...,0.100400,0.070900,0.039000,0.035200,0.044700,0.039400,0.035500,0.044000,0.036400,0.043900


It appears we only have 208 samples to work with, which is small.

Let's copy the target values and see how many of each there are.

In [5]:
y = df.values[:, 60]

df[60].value_counts()

M    111
R     97
Name: 60, dtype: int64

111 mines and 97 rocks.

Now, let's get the features.

In [6]:
X = df.values[:, 0:60].astype(float)

X.shape

(208, 60)

As our target values are nominal, we must first convert them into ordinal values (0 and 1).

In [7]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)

np.unique(y)

array([0, 1])

Let's begin composing our neural network.

Using Keras, our first network will take in the 60 sonar energy values as inputs to 60 corresponding ReLus.

The output neuron will have a sigmoid activation, due to the binary nature of the classification problem. Apparently, a loss function of binary cross-entropy is preferred for binary classification problems as well, and is therefore made use of.

In [8]:
from keras.models import Sequential
from keras.layers import Dense

def net1():
    model = Sequential([
        Dense(60, input_dim=60, init="normal", activation="relu"),
        Dense(1, init="normal", activation="sigmoid")
    ])
    
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    return model

In [9]:
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score

clf = KerasClassifier(build_fn=net1, nb_epoch=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)

res1 = cross_val_score(clf, X, y, cv=kfold)

In [10]:
print(res1)
print("Results: %.2f%% (%.2f%%)" % (res1.mean()*100, res1.std()*100))

[ 0.86363637  0.80952382  0.80952381  0.85714287  0.85714287  0.85714287
  0.76190477  0.90000001  0.80000001  0.65000001]
Results: 81.66% (6.72%)


Let's tune our data with some more feature engineering.

In [11]:
from sklearn.pipeline import Pipeline

clfs = []
clfs.append(("standardize", StandardScaler()))
clfs.append(("mlp", KerasClassifier(build_fn=net1, nb_epoch=100, batch_size=5, verbose=0)))
pipe = Pipeline(clfs)

res2 = cross_val_score(pipe, X, y, cv=kfold)

In [12]:
print(res2)
print("Standardized: %.2f%% (%.2f%%)" % (res2.mean()*100, res2.std()*100))

[ 0.81818183  0.85714287  0.80952382  0.90476191  0.95238096  0.90476191
  0.76190478  0.85000001  0.85000001  0.85000001]
Standardized: 85.59% (5.16%)


Let's experiment with the network topology. Seeing the effects of decreasing the model size and increasing it will enable us to better tune it.

In [13]:
def sm_net():
    model = Sequential([
        Dense(30, input_dim=60, init="normal", activation="relu"),
        Dense(1, init="normal", activation="sigmoid")
    ])
    
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

clfs = []
clfs.append(("standardize", StandardScaler()))
clfs.append(("mlp", KerasClassifier(build_fn=sm_net, nb_epoch=100, batch_size=5, verbose=0)))
pipe = Pipeline(clfs)

res3 = cross_val_score(pipe, X, y, cv=kfold)

In [14]:
print(res3)
print("Smaller: %.2f%% (%.2f%%)" % (res3.mean()*100, res3.std()*100))

[ 0.86363637  0.90476191  0.80952382  0.85714287  0.85714287  0.95238096
  0.80952382  0.90000001  0.85000001  0.80000001]
Smaller: 86.04% (4.58%)


In [15]:
def lg_net():
    model = Sequential([
        Dense(60, input_dim=60, init="normal", activation="relu"),
        Dense(30, init="normal", activation="relu"),
        Dense(1, init="normal", activation="sigmoid")
    ])
    
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

clfs = []
clfs.append(("standardize", StandardScaler()))
clfs.append(("mlp", KerasClassifier(build_fn=lg_net, nb_epoch=100, batch_size=5, verbose=0)))
pipe = Pipeline(clfs)

res4 = cross_val_score(pipe, X, y, cv=kfold)

In [16]:
print(res4)
print("Larger: %.2f%% (%.2f%%)" % (res4.mean()*100, res4.std()*100))

[ 0.90909091  0.85714287  0.76190478  0.85714287  0.85714287  0.85714287
  0.76190478  0.85000001  0.85000001  0.85000001]
Larger: 84.11% (4.29%)


Let's try Stochastic Gradient Descent without Nesterov Momentum as an optimizer.

We'll apply it to the deeper model we just constructed.

In [17]:
from keras.layers import Dropout
from keras.constraints import maxnorm
from keras.optimizers import SGD

def net2():
    model = Sequential([
        Dropout(0.2, input_shape=(60,)),
        Dense(60, init="normal", activation="relu", W_constraint=maxnorm(3)),
        Dense(30, init="normal", activation="relu", W_constraint=maxnorm(3)),
        Dense(1, init="normal", activation="sigmoid")
    ])
    
    sgd = SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=False)
    model.compile(loss="binary_crossentropy", optimizer=sgd, metrics=["accuracy"])
    return model

clfs = []
clfs.append(("standardize", StandardScaler()))
clfs.append(("mlp", KerasClassifier(build_fn=net2, nb_epoch=100, batch_size=5, verbose=0)))
pipe = Pipeline(clfs)

res5 = cross_val_score(pipe, X, y, cv=kfold)

In [18]:
print(res5)
print("SGD Accuracy: %.2f%% (%.2f%%)" % (res5.mean()*100, res5.std()*100))

[ 0.95454546  0.80952382  0.85714287  0.85714287  0.80952382  0.80952381
  0.76190478  0.8         0.80000001  0.8       ]
SGD Accuracy: 82.59% (5.04%)


Now let's try with Nesterov Momentum.

In [19]:
def net3():
    model = Sequential([
        Dropout(0.2, input_shape=(60,)),
        Dense(60, init="normal", activation="relu", W_constraint=maxnorm(3)),
        Dense(30, init="normal", activation="relu", W_constraint=maxnorm(3)),
        Dense(1, init="normal", activation="sigmoid")
    ])
    
    sgd = SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=True)
    model.compile(loss="binary_crossentropy", optimizer=sgd, metrics=["accuracy"])
    return model

clfs = []
clfs.append(("standardize", StandardScaler()))
clfs.append(("mlp", KerasClassifier(build_fn=net3, nb_epoch=100, batch_size=5, verbose=0)))
pipe = Pipeline(clfs)

res6 = cross_val_score(pipe, X, y, cv=kfold)

In [20]:
print(res6)
print("Nesterov Momentum Accuracy: %.2f%% (%.2f%%)" % (res6.mean()*100, res6.std()*100))

[ 0.90909091  0.85714287  0.85714287  0.85714287  0.85714287  0.90476191
  0.85714287  0.8         0.90000001  0.80000001]
Nesterov Momentum Accuracy: 86.00% (3.64%)


For the next model, we'll place our Dropout layers intermittently.

In [21]:
def net4():
    model = Sequential([
        Dense(60, input_dim=60, init="normal", activation="relu", W_constraint=maxnorm(3)),
        Dropout(0.2),
        Dense(30, init="normal", activation="relu", W_constraint=maxnorm(3)),
        Dropout(0.2),
        Dense(1, init="normal", activation="sigmoid")
    ])
    
    sgd = SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=True)
    model.compile(loss="binary_crossentropy", optimizer=sgd, metrics=["accuracy"])
    return model

clfs = []
clfs.append(("standardize", StandardScaler()))
clfs.append(("mlp", KerasClassifier(build_fn=net3, nb_epoch=100, batch_size=5, verbose=0)))
pipe = Pipeline(clfs)

res7 = cross_val_score(pipe, X, y, cv=kfold)

In [22]:
print(res7)
print("Multi-Dropout Accuracy: %.2f%% (%.2f%%)" % (res7.mean()*100, res7.std()*100))

[ 0.90909091  0.85714287  0.80952381  0.90476191  0.80952382  0.80952381
  0.71428572  0.8         0.90000001  0.85000001]
Multi-Dropout Accuracy: 83.64% (5.75%)


## Conclusions ##

It seems that our best model is `net3` with an accuracy of 86% and a variation of 3.64%. This is the best accuracy and lowest variation, which is preferred.

It seemed that Nesterov Momentum with a larger network is more effective than smaller networks that use SGD or Adam as optimizers.